In [1]:
import requests
import json
import pandas as pd
import string
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
from collections import Counter

In [77]:
def getPlayerContractData(df_players,cap_table):
    """
    Returns a dictionary with key corresponding to player and value corresponding to dictionary of two tables:
    The first table is contract specific data and player-immutable data, and the second table is advanced player
    statistics broken down by year
    """
    
    dict_contracts = {}
    
    for p in list(df_players.index):
        first,last = p.split(' ')[0],p.split(' ')[1]
        url = f'https://www.capfriendly.com/players/{first}-{last}'
        response = requests.get(url)
        try:
            tables = pd.read_html(response.text)
        except ValueError:
            continue
        
        if len(tables) < 2:
            continue
        dict_player = {}
        df_allContracts = pd.DataFrame()
        for t in (tables[:-1]):
            t_new = t.iloc[1:,[0,2]]
            contract_length = t_new.shape[0]-1
            start_year = (int(t_new.iloc[0,0].split('-')[0]) if isinstance(t_new.iloc[0,0].split('-')[0],str) 
                          else None)
            total_value = (int(t_new.iloc[-1,1].replace('$','').replace(',','')) if isinstance(t_new.iloc[-1,1],str) 
                           else None)
            df_allContracts.loc[start_year,'Value'] = total_value
            df_allContracts.loc[start_year,'Length'] = contract_length

        df_allContracts = df_allContracts.dropna(axis=0)
        if (df_allContracts.shape[0] < 1):
            continue
        df_allContracts['AAV'] = (df_allContracts['Value']/df_allContracts['Length'])
        for y in list(df_allContracts.index):
            df_allContracts.loc[y,'Contract Age'] = y-df_players.loc[p,'birthYear']
            if y >= 2005:
                df_allContracts.loc[y,'Cap Hit'] = df_allContracts.loc[y,'AAV']/cap_table.loc[y,'UPPER LIMIT']
            else:
                df_allContracts.loc[y,'Cap Hit'] = 0
        df_allContracts = df_allContracts[df_allContracts['Cap Hit'] > 0]
        
        df_allContracts['Position'] = df_players.loc[p,'Position']
        df_allContracts['Nationality'] = df_players.loc[p,'nationality']
        df_allContracts['Height'] = df_players.loc[p,'height']
        df_allContracts['Overall Pick'] = df_players.loc[p,'Overall Pick']
        
        summary_table = tables[-1]
        summary_table = summary_table[~summary_table['SEASON'].isna()].iloc[:-1,:]
        summary_table = summary_table[summary_table['LEAGUE'] == 'NHL']
        summary_table['SEASON ENDING'] = (summary_table['SEASON'].apply(lambda x: int(x.split('-')[0][:2] + 
                                                                                      x.split('-')[1])))
        summary_table = summary_table.set_index('SEASON ENDING')

        dict_player['Summary'] = df_allContracts
        dict_player['Stats'] = summary_table
        dict_contracts[p] = dict_player
        
    return dict_contracts

In [24]:
def getDataTable_g(dict_players):
    """
    Returns a table with contract data that can be further processed for modeling use
    Specifically returns a table for modeling goalie contracts
    """
    
    df_final = pd.DataFrame()
    
    for p in list(dict_players.keys()):
        
        df_stat_summary = dict_players[p]['Stats'].drop(columns=['SEASON','TEAM','LEAGUE','Unnamed: 3','Unnamed: 7',
                                                                 'Unnamed: 15','PLAYOFFS.1','PLAYOFFS'])
        df_stat_summary.columns = ['GAMES_PLAYED','GAA','Sv%','GP_PLAYOFF','GAA_PLAYOFF','Sv%_PLAYOFF','GA60',
                                   'xGA60','GSAx60','GA60_PLAYOFF','xGA60_PLAYOFF','GSAx60_PLAYOFF']
        df_stat_summary = df_stat_summary.fillna(0)
        df_stat_summary = df_stat_summary.applymap(lambda x: pd.to_numeric(x) if x != '-' else 0)

        df_contract_allData = dict_players[p]['Summary'].copy()

        df_data = pd.DataFrame()

        for y in list(dict_players[p]['Summary'].index):
            ct_years = [y,y-1,y-2]
            ct_year_symbols = ['_t','_t1','_t2']
            ls_temp = []
            t_count = 0
            for t in ct_years:
                if t in df_stat_summary.index:
                    row_t = df_stat_summary.loc[t]
                    row_t.index = [c + ct_year_symbols[ct_years.index(t)] for c in row_t.index]
                    df_temp = pd.DataFrame(row_t).T
                    df_temp.index = [y]
                    ls_temp.append(df_temp)
                    t_count += 1
                else:
                    df_temp = pd.DataFrame(0,columns = [c + ct_year_symbols[ct_years.index(t)] for c in 
                                                        df_stat_summary.columns],index=[y])
                    ls_temp.append(df_temp)

            if t_count > 0:
                avg = (np.array(ls_temp[0].values)+np.array(ls_temp[1].values)+(ls_temp[2].values))/t_count
                avg_columns = [c + '_avg' for c in df_stat_summary.columns]
                avg_df = pd.DataFrame(avg,columns=avg_columns,index=[y])
                df_y = pd.concat([ls_temp[0],ls_temp[1],ls_temp[2],avg_df],axis=1)

                if df_data.empty:
                    df_data = df_y
                else:
                    df_data = pd.concat([df_data,df_y],axis=0)

            else:
                df_contract_allData = df_contract_allData.drop(labels=[y],axis=0)

        df_final_player = df_contract_allData.merge(df_data,left_index=True,right_index=True)
        
        if df_final.empty:
            df_final = df_final_player
        else:
            df_final = pd.concat([df_final,df_final_player],axis=0)
    
    return df_final

In [80]:
def getDataTable_ng(dict_contracts):
    """
    Returns a table with contract data that can be further processed for modeling use
    Specifically returns a table for modeling non-goalie contracts
    """
    
    df_final = pd.DataFrame()
    
    for p in list(dict_contracts.keys()):
        df_stat_summary = dict_contracts[p]['Stats'].drop(columns=['SEASON','TEAM','LEAGUE','Unnamed: 3',
                                                                   'Unnamed: 10','Unnamed: 25','Unnamed: 33',
                                                                   'PLAYOFFS','G.3','G.2'])
        df_stat_summary.columns = ['GAMES_PLAYED','G','A','P','PLUS_MINUS','PIM','GP_PLAYOFF','G_PLAYOFF','A_PLAYOFF',
                                   'P_PLAYOFF','PLUS_MINUS_PLAYOFF','PIM_PLAYOFF','TOI','ixG','xG±/60','RelxG±/60',
                                   'C±/60', 'RelC±/60','TOI_PLAYOFF','ixG_PLAYOFF','xG±/60_PLAYOFF',
                                   'RelxG±/60_PLAYOFF','C±/60_PLAYOFF','RelC±/60_PLAYOFF']
        df_stat_summary = df_stat_summary.fillna(0)
        df_stat_summary['TOI'] = df_stat_summary['TOI'].apply(lambda x: (x.split(':')))
        df_stat_summary['TOI'] = df_stat_summary['TOI'].apply(lambda x: int(x[0])+(pd.to_numeric(x[1])/60) 
                                                              if len(x) > 1 else 0)
        df_stat_summary['TOI_PLAYOFF'] = df_stat_summary['TOI_PLAYOFF'].apply(lambda x: (x.split(':')))
        df_stat_summary['TOI_PLAYOFF'] = df_stat_summary['TOI_PLAYOFF'].apply(lambda x: int(x[0]) + 
                                                                              (pd.to_numeric(x[1])/60) 
                                                                              if len(x) > 1 else 0)
        df_stat_summary = df_stat_summary.applymap(lambda x: pd.to_numeric(x) if x != '-' else 0)

        df_stat_summary['G_PG'] = df_stat_summary.apply(lambda row: (row.G/row.GAMES_PLAYED 
                                                                     if row.GAMES_PLAYED > 0 else 0),axis=1)
        df_stat_summary['P_PG'] = df_stat_summary.apply(lambda row: (row.P/row.GAMES_PLAYED 
                                                                     if row.GAMES_PLAYED > 0 else 0),axis=1)
        df_stat_summary['PIM_PG'] = df_stat_summary.apply(lambda row: (row.PIM/row.GAMES_PLAYED 
                                                                       if row.GAMES_PLAYED > 0 else 0),axis=1)

        df_stat_summary['G_PG_PLAYOFF'] = df_stat_summary.apply(lambda row: (row.G_PLAYOFF/row.GP_PLAYOFF 
                                                                             if row.GP_PLAYOFF > 0 else 0),axis=1)
        df_stat_summary['P_PG_PLAYOFF'] = df_stat_summary.apply(lambda row: (row.P_PLAYOFF/row.GP_PLAYOFF 
                                                                             if row.GP_PLAYOFF > 0 else 0),axis=1)
        df_stat_summary = df_stat_summary.applymap(lambda x: pd.to_numeric(x) if x != '-' else 0)
        
        df_contract_allData = dict_contracts[p]['Summary'].copy()

        df_data = pd.DataFrame()

        for y in list(dict_contracts[p]['Summary'].index):
            ct_years = [y,y-1,y-2]
            ct_year_symbols = ['_t','_t1','_t2']
            ls_temp = []
            t_count = 0
            for t in ct_years:
                if t in df_stat_summary.index:
                    row_t = df_stat_summary.loc[t]
                    row_t.index = [c + ct_year_symbols[ct_years.index(t)] for c in row_t.index]
                    df_temp = pd.DataFrame(row_t).T
                    df_temp.index = [y]
                    ls_temp.append(df_temp)
                    t_count += 1
                else:
                    df_temp = pd.DataFrame(0,columns = [c + ct_year_symbols[ct_years.index(t)] 
                                                        for c in df_stat_summary.columns],index=[y])
                    ls_temp.append(df_temp)

            if t_count > 0:
                avg = (np.array(ls_temp[0].values)+np.array(ls_temp[1].values)+(ls_temp[2].values))/t_count
                avg_columns = [c + '_avg' for c in df_stat_summary.columns]
                avg_df = pd.DataFrame(avg,columns=avg_columns,index=[y])
                df_y = pd.concat([ls_temp[0],ls_temp[1],ls_temp[2],avg_df],axis=1)

                if df_data.empty:
                    df_data = df_y
                else:
                    df_data = pd.concat([df_data,df_y],axis=0)

            else:
                df_contract_allData = df_contract_allData.drop(labels=[y],axis=0)

        df_final_player = df_contract_allData.merge(df_data,left_index=True,right_index=True)
        
        if df_final.empty:
            df_final = df_final_player
        else:
            df_final = pd.concat([df_final,df_final_player],axis=0)
    
    return df_final

Since the 2004-2005 season, the NHL has used a hard salary cap limit to dictate newly signed player contracts. To obtain contract data, I will look at players who have been active since 2006 to balance getting as much historical data as possible while also staying within the context of how teams and contracts are currently constructed. To do this, I will go through the rosters of current NHL teams in the API and scrape the player data.

In [4]:
# create api request for teams
requests_teams = requests.get('https://statsapi.web.nhl.com/api/v1/teams')
teams = json.loads(requests_teams.text)['teams']

In [8]:
# current number of teams
len(teams)

32

In [10]:
# example of team entry in NHL API
teams[0]

{'id': 1,
 'name': 'New Jersey Devils',
 'link': '/api/v1/teams/1',
 'venue': {'name': 'Prudential Center',
  'link': '/api/v1/venues/null',
  'city': 'Newark',
  'timeZone': {'id': 'America/New_York', 'offset': -4, 'tz': 'EDT'}},
 'abbreviation': 'NJD',
 'teamName': 'Devils',
 'locationName': 'New Jersey',
 'firstYearOfPlay': '1982',
 'division': {'id': 18,
  'name': 'Metropolitan',
  'nameShort': 'Metro',
  'link': '/api/v1/divisions/18',
  'abbreviation': 'M'},
 'conference': {'id': 6, 'name': 'Eastern', 'link': '/api/v1/conferences/6'},
 'franchise': {'franchiseId': 23,
  'teamName': 'Devils',
  'link': '/api/v1/franchises/23'},
 'shortName': 'New Jersey',
 'officialSiteUrl': 'http://www.newjerseydevils.com/',
 'franchiseId': 23,
 'active': True}

In [12]:
# get list of teams and IDs to loop through API
team_ids = {t['name']:t['id'] for t in teams}

In [13]:
team_ids

{'New Jersey Devils': 1,
 'New York Islanders': 2,
 'New York Rangers': 3,
 'Philadelphia Flyers': 4,
 'Pittsburgh Penguins': 5,
 'Boston Bruins': 6,
 'Buffalo Sabres': 7,
 'Montréal Canadiens': 8,
 'Ottawa Senators': 9,
 'Toronto Maple Leafs': 10,
 'Carolina Hurricanes': 12,
 'Florida Panthers': 13,
 'Tampa Bay Lightning': 14,
 'Washington Capitals': 15,
 'Chicago Blackhawks': 16,
 'Detroit Red Wings': 17,
 'Nashville Predators': 18,
 'St. Louis Blues': 19,
 'Calgary Flames': 20,
 'Colorado Avalanche': 21,
 'Edmonton Oilers': 22,
 'Vancouver Canucks': 23,
 'Anaheim Ducks': 24,
 'Dallas Stars': 25,
 'Los Angeles Kings': 26,
 'San Jose Sharks': 28,
 'Columbus Blue Jackets': 29,
 'Minnesota Wild': 30,
 'Winnipeg Jets': 52,
 'Arizona Coyotes': 53,
 'Vegas Golden Knights': 54,
 'Seattle Kraken': 55}

In [27]:
# get roster for each team
dict_teamRosters = {}

for team in list(team_ids.keys()):
    team_id = team_ids[team]
    season = 2006
    list_rosters = []
    while (season < 2022):
        season_string = f'{season}{season+1}'
        request_roster_season = requests.get(f'https://statsapi.web.nhl.com/api/v1/teams/{team_id}/roster?expand=team.roster&season={season_string}')
        roster = json.loads(request_roster_season.text)
        df_roster = pd.DataFrame()
        try:
            for r in roster['roster']:
                player_id = r['person']['id']
                df_roster.loc[player_id,'Name'] = r['person']['fullName']
                df_roster.loc[player_id,'Position'] = r['position']['code']
            list_rosters.append(df_roster)
            season += 1
        except KeyError:
            season += 1
    df_team = pd.concat(list_rosters,axis=0)
    df_team = df_team.drop_duplicates()
    dict_teamRosters[team] = df_team

In [28]:
# example of a team roster
dict_teamRosters['New Jersey Devils']

,Name,Position
8456100,Jim Dowd,C
8458522,Richard Matvichuk,D
8458978,Sergei Brylin,L
8459454,Jay Pandolfo,L
8459457,Jamie Langenbrunner,R
...,...,...
8473541,Jonathan Bernier,G
8476903,Jon Gillies,G
8477202,Andrew Hammond,G
8481033,Akira Schmid,G


Since we are looking at all team rosters since 2006, we expect there to be some overlap because players get traded between teams or are signed to new contracts in free agency. Since there is a certain amount of data that will not change with a player's team or over time, such as height, birth date, nationality, etc., we will combine the rosters and remove any duplicates before using the API to obtain this data.

In [36]:
# combine roster dataframes into a single large dataframe for all players
df_players = pd.DataFrame()
for df_team in list(dict_teamRosters.values()):
    if df_players.empty:
        df_players = df_team
    else:
        df_players = pd.concat([df_players,df_team],axis=0)

In [37]:
df_players.head()

,Name,Position
8456100,Jim Dowd,C
8458522,Richard Matvichuk,D
8458978,Sergei Brylin,L
8459454,Jay Pandolfo,L
8459457,Jamie Langenbrunner,R


In [38]:
df_players.shape

(6802, 2)

In [39]:
# remove any duplicate players by ID
df_players['player_id'] = df_players.index
df_players = df_players.drop_duplicates(subset=['Name','player_id'])

In [40]:
df_players.shape

(3167, 3)

In [41]:
# remove player_id column
df_players = df_players.drop(columns=['player_id'],axis=1)

Now that we have a list of players on whom to find contracts, we can append relevant information from the API for every player. This data that we store here will be immutable, or applicable to the player over the course of their career, whereas other data, such as recent statistics, will change over the course of a player's career. We do, however, make the assumption that a player's height will not change during their career, which is reasonable because players must be 18 to play professionally in the NHL, and it is reasonable to conclude that a player is done growing by that age. We do not, however, make this assumption about weight.

In [42]:
# get immutable player data
for i in list(df_players.index):
    player_request = requests.get(f'https://statsapi.web.nhl.com/api/v1/people/{i}')
    player = json.loads(player_request.text)['people'][0]
    df_players.loc[i,'birthDate'] = player['birthDate']
    df_players.loc[i,'nationality'] = player['nationality']
    height_list = [int(player['height'].split(' ')[i].translate(str.maketrans('','',string.punctuation))) 
                   for i in [0,1]]
    df_players.loc[i,'height'] = height_list[0]*12+height_list[1]

In [43]:
df_players.head()

,Name,Position,birthDate,nationality,height
8456100,Jim Dowd,C,1968-12-25,USA,72.0
8458522,Richard Matvichuk,D,1973-02-05,CAN,75.0
8458978,Sergei Brylin,L,1974-01-13,RUS,70.0
8459454,Jay Pandolfo,L,1974-12-27,USA,73.0
8459457,Jamie Langenbrunner,R,1975-07-24,USA,73.0


In [44]:
# name index
df_players.index.name = 'Player ID'

Even though we removed duplicate player IDs before, we will double check for duplicates and missing values.

In [45]:
# check for missing data
df_players.isna().sum()

Name           0
Position       0
birthDate      0
nationality    0
height         0
dtype: int64

In [46]:
# check for duplicates - this is done by index (player ID)
df_players.duplicated().value_counts()

False    3167
dtype: int64

In [569]:
# confirm shape of dataframe
df_players.shape

(3167, 5)

In [48]:
# check for duplicate names
df_players.duplicated(subset='Name').sum()

9

Despite corresponding to different player IDs, there are several players who have identical names; while this is purely coincidental, when it comes to web-scraping, there will be some difficulty in determining what the player's corresponding URL should be, so for ease of implementation, those contracts will be disregarded.

In [50]:
# find duplicate names of players
bad_names = [name for name, count in Counter(df_players['Name']).items() if count != 1]

In [51]:
# drop players with same names from the dataframe
df_players = df_players[~df_players['Name'].isin(bad_names)]

In [52]:
# get new size of dataframe
df_players.shape

(3150, 5)

A player's draft record is also something that does not change over the course of the career - in fact it only happens once - so we will also populate the players dataframe with that information. For the rare case of undrafted players, there is some question of how they should be treated - without knowing if this is even a significant predictor, we will treat the missing values by filling with the median draft pick.

In [54]:
# get draft data
start_year = 1990
current_year = 2023

df_draft = pd.DataFrame()

while (start_year < current_year):
    draft_req = requests.get(f'https://statsapi.web.nhl.com/api/v1/draft/{start_year}')
    draft = json.loads(draft_req.text)['drafts'][0]
    for r in (draft['rounds']):
        for p in (r['picks']):
            player = p['prospect']['fullName']
            pick_overall = p['pickOverall']
            df_draft.loc[player,'Overall Pick'] = pick_overall
            df_draft.loc[player,'Year'] = start_year
    start_year += 1

In [55]:
df_draft.head()

,Overall Pick,Year
Mika Stromberg,211.0,1990.0
Tyler Ertel,212.0,1990.0
Brett Larson,213.0,1990.0
Tommy Soderstrom,214.0,1990.0
Michael Thompson,215.0,1990.0


In [56]:
df_draft.shape

(7804, 2)

In [57]:
# check for missing values
df_draft.isna().sum()

Overall Pick    0
Year            0
dtype: int64

In [58]:
# reset index to match draft data to player names
df_players = df_players.reset_index().set_index('Name')

In [59]:
df_players.head()

,Player ID,Position,birthDate,nationality,height
Name,,,,,
Jim Dowd,8456100,C,1968-12-25,USA,72.0
Richard Matvichuk,8458522,D,1973-02-05,CAN,75.0
Sergei Brylin,8458978,L,1974-01-13,RUS,70.0
Jay Pandolfo,8459454,L,1974-12-27,USA,73.0
Jamie Langenbrunner,8459457,R,1975-07-24,USA,73.0


In [60]:
# in case of any issues with casing in player names, switch all draft names to lowercase
df_draft_lower = df_draft.copy()
df_draft_lower.index = pd.Series(df_draft.index).apply(lambda x: x.lower())

for p in list(df_players.index):
    p_lower = p.lower()
    if p_lower in df_draft_lower.index:
        df_players.loc[p,'Overall Pick'] = df_draft_lower.loc[p_lower,'Overall Pick']
    else:
        df_players.loc[p,'Overall Pick'] = df_draft_lower['Overall Pick'].median()

In [61]:
df_players.head()

,Player ID,Position,birthDate,nationality,height,Overall Pick
Name,,,,,,
Jim Dowd,8456100,C,1968-12-25,USA,72.0,121.0
Richard Matvichuk,8458522,D,1973-02-05,CAN,75.0,8.0
Sergei Brylin,8458978,L,1974-01-13,RUS,70.0,42.0
Jay Pandolfo,8459454,L,1974-12-27,USA,73.0,32.0
Jamie Langenbrunner,8459457,R,1975-07-24,USA,73.0,35.0


In [62]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3150 entries, Jim Dowd to Matty Beniers
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Player ID     3150 non-null   int64  
 1   Position      3150 non-null   object 
 2   birthDate     3150 non-null   object 
 3   nationality   3150 non-null   object 
 4   height        3150 non-null   float64
 5   Overall Pick  3150 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 236.8+ KB


Since the date of birth feature is in string form, we need to convert this to a number that can be used further in the data-cleaning process.

In [64]:
df_players['birthYear'] = df_players['birthDate'].apply(lambda x: pd.to_numeric(x.split('-')[0]))

In [69]:
df_players.head()

,Player ID,Position,birthDate,nationality,height,Overall Pick,birthYear
Name,,,,,,,
Jim Dowd,8456100,C,1968-12-25,USA,72.0,121.0,1968
Richard Matvichuk,8458522,D,1973-02-05,CAN,75.0,8.0,1973
Sergei Brylin,8458978,L,1974-01-13,RUS,70.0,42.0,1974
Jay Pandolfo,8459454,L,1974-12-27,USA,73.0,32.0,1974
Jamie Langenbrunner,8459457,R,1975-07-24,USA,73.0,35.0,1975


In [65]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3150 entries, Jim Dowd to Matty Beniers
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Player ID     3150 non-null   int64  
 1   Position      3150 non-null   object 
 2   birthDate     3150 non-null   object 
 3   nationality   3150 non-null   object 
 4   height        3150 non-null   float64
 5   Overall Pick  3150 non-null   float64
 6   birthYear     3150 non-null   int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 261.4+ KB


Now that we have constructed a dataframe for players and some basic data, we need to obtain the salary cap data for each year to determine the contract value in terms of the salary cap.

In [73]:
# get salary cap data
url_cap = 'https://www.capfriendly.com/salary-cap'
response_cap = requests.get(url_cap)
tables_cap = pd.read_html(response_cap.text)[0]
tables_cap['SEASON'] = tables_cap['SEASON'].apply(lambda x: int(x.split('-')[0]))
cap_table = tables_cap.set_index('SEASON').drop(columns=['CONFIRMED','% CHANGE']).applymap(lambda x: 
            int(x.replace('$','').replace(',','')))

In [74]:
cap_table

,UPPER LIMIT,LOWER LIMIT,MIN. SALARY
SEASON,,,
2025,92000000,68000000,775000
2024,87500000,64700000,775000
2023,83500000,61700000,775000
2022,82500000,61000000,750000
2021,81500000,60200000,750000
2020,81500000,60200000,700000
2019,81500000,60200000,700000
2018,79500000,58800000,650000
2017,75000000,55400000,650000


Now we can use defined functions to combine and clean data that will be used to make the dataframe on which the models can be run. To do this, we need to split the players into goalies and non-goalies - this is because goalie metrics are an entirely diffrent set of statistics.

In [70]:
# split the player dataframe into goalie and non-goalie players
df_players_g = df_players[df_players['Position'] == 'G']
df_players_ng = df_players[df_players['Position'] != 'G']

In [71]:
# confirm that only goalies appear in goalie set
df_players_g['Position'].unique()

array(['G'], dtype=object)

In [72]:
# confirm that no goalies appear in non-goalie set
df_players_ng['Position'].unique()

array(['C', 'D', 'L', 'R'], dtype=object)

In [25]:
# get player contract data for goalies
dict_contracts_g = getPlayerContractData(df_players_g,cap_table)

In [26]:
# construct dataframe with statistics for goalies
dataTable_g = getDataTable_g(dict_contracts_g)

In [27]:
dataTable_g.head()

,Value,Length,AAV,Contract Age,Cap Hit,Position,Nationality,Height,Overall Pick,GAMES_PLAYED_t,...,Sv%_avg,GP_PLAYOFF_avg,GAA_PLAYOFF_avg,Sv%_PLAYOFF_avg,GA60_avg,xGA60_avg,GSAx60_avg,GA60_PLAYOFF_avg,xGA60_PLAYOFF_avg,GSAx60_PLAYOFF_avg
2021,8250000.0,2.0,4125000.0,33.0,0.050613,G,CAN,72.0,11.0,24.0,...,0.908333,0.000000,0.000000,0.000000,2.643333,2.640000,-0.003333,0.00,0.000000,0.000000
2011,2500000.0,2.0,1250000.0,23.0,0.019440,G,CAN,72.0,11.0,25.0,...,0.935000,0.000000,0.000000,0.000000,1.755000,2.155000,0.400000,0.00,0.000000,0.000000
2013,5800000.0,2.0,2900000.0,25.0,0.045101,G,CAN,72.0,11.0,14.0,...,0.914667,0.333333,0.000000,0.333333,1.990000,1.916667,-0.073333,0.00,0.236667,0.236667
2015,8300000.0,2.0,4150000.0,27.0,0.058123,G,CAN,72.0,11.0,58.0,...,0.918667,0.333333,0.000000,0.333333,2.083333,2.273333,0.190000,0.00,0.236667,0.236667
2017,2750000.0,1.0,2750000.0,29.0,0.036667,G,CAN,72.0,11.0,39.0,...,0.911667,1.333333,1.096667,0.291000,2.473333,2.356667,-0.116667,0.95,0.823333,-0.126667


In [78]:
# get player contract data for non-goalies
dict_contracts_ng = getPlayerContractData(df_players_ng,cap_table)

In [81]:
# construct dataframe with statistics for non-goalies
dataTable_ng = getDataTable_ng(dict_contracts_ng)

In [82]:
dataTable_ng.shape

(5648, 125)

In [83]:
dataTable_ng.head()

,Value,Length,AAV,Contract Age,Cap Hit,Position,Nationality,Height,Overall Pick,GAMES_PLAYED_t,...,ixG_PLAYOFF_avg,xG±/60_PLAYOFF_avg,RelxG±/60_PLAYOFF_avg,C±/60_PLAYOFF_avg,RelC±/60_PLAYOFF_avg,G_PG_avg,P_PG_avg,PIM_PG_avg,G_PG_PLAYOFF_avg,P_PG_PLAYOFF_avg
2008,7500000.0,3.0,2500000.0,34.0,0.044092,L,USA,73.0,32.0,54.0,...,0.203333,-0.143333,-0.093333,-3.380000,-1.830000,0.167570,0.339205,0.233363,0.067340,0.215488
2011,600000.0,1.0,600000.0,37.0,0.009331,L,USA,73.0,32.0,0.0,...,0.310000,-0.080000,-0.055000,-3.365000,0.485000,0.079445,0.168506,0.139660,0.071429,0.071429
2012,600000.0,1.0,600000.0,38.0,0.010000,L,USA,73.0,32.0,62.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.046526,0.110732,0.122208,0.000000,0.000000
2010,833334.0,1.0,833334.0,36.0,0.014029,L,USA,73.0,32.0,52.0,...,0.410000,-0.196667,-0.130000,-5.623333,-1.506667,0.127038,0.260485,0.228909,0.047619,0.047619
2005,1655130.0,1.0,1655130.0,30.0,0.042439,R,USA,73.0,35.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.235365,0.597847,0.822327,0.229167,0.575000


In [87]:
# export data to csv for use in modeling notebook
dataTable_ng.to_csv('data_ng.csv',index=False)

Now that we have the data we are going to use for modelling, we can similarly generate the same data for unrestricted free agents for whom we want to project contract values. First, we need to get a list of unrestricted free agents.

In [89]:
# get list of new UFAs
url_ufa = 'https://www.spotrac.com/nhl/free-agents/ufa/'
response_ufa = requests.get(url_ufa)
tables_ufa = pd.read_html(response_ufa.text)[0]
list_players = list(tables_ufa.iloc[:,0])

In [108]:
len(list_players)

267

In [92]:
list_players[0:10]

['Patrick Kane',
 'Jonathan Toews',
 "Ryan O'Reilly",
 'Vladimir Tarasenko',
 'James Van Riemsdyk',
 'John Klingberg',
 'Max Pacioretty',
 'Sean Monahan',
 'Milan Lucic',
 'Mathew Dumba']

In [96]:
# get new dataframe for UFAs
df_players_new = df_players_ng[df_players_ng.index.isin(list_players)]

In [97]:
df_players_new.head()

,Player ID,Position,birthDate,nationality,height,Overall Pick,birthYear
Name,,,,,,,
Zach Parise,8470610,L,1984-07-28,USA,71.0,17.0,1984
Damon Severson,8476923,D,1994-08-07,CAN,74.0,60.0,1994
John Moore,8475186,D,1990-11-19,USA,75.0,21.0,1990
Miles Wood,8477425,L,1995-09-13,USA,74.0,100.0,1995
Steven Santini,8477463,D,1995-03-07,USA,74.0,42.0,1995


Now that we have the dataframe of players and their relevant information, we can re-configure a defined function to get a matching table for each player - using the contract year 2023 to get the relevant statistics.

In [98]:
def getPlayerContractData_newContracts(df_players,contract_year):
    """
    Returns a dictionary with key corresponding to player and value corresponding to dictionary of two tables:
    The first table is contract specific data and player-immutable data, and the second table is advanced player
    statistics broken down by year
    """
    dict_contracts = {}
    
    for p in list(df_players.index):
        first,last = p.split(' ')[0],p.split(' ')[1]
        url = f'https://www.capfriendly.com/players/{first}-{last}'
        response = requests.get(url)
        try:
            tables = pd.read_html(response.text)
        except ValueError:
            continue
        
        if len(tables) < 2:
            continue
        dict_player = {}
        df_newContract = pd.DataFrame()
        
        y = contract_year
        df_newContract.loc[y,'Name'] = p
        df_newContract.loc[y,'ContractAge'] = y-df_players.loc[p,'birthYear']
        
        df_newContract['Position'] = df_players.loc[p,'Position']
        df_newContract['Nationality'] = df_players.loc[p,'nationality']
        df_newContract['Height'] = df_players.loc[p,'height']
        df_newContract['Overall Pick'] = df_players.loc[p,'Overall Pick']
        
        summary_table = tables[-1]
        summary_table = summary_table[~summary_table['SEASON'].isna()].iloc[:-1,:]
        summary_table = summary_table[summary_table['LEAGUE'] == 'NHL']
        summary_table['SEASON ENDING'] = (summary_table['SEASON'].apply(lambda x: int(x.split('-')[0][:2] + 
                                                                                      x.split('-')[1])))
        summary_table = summary_table.set_index('SEASON ENDING')

        dict_player['Summary'] = df_newContract
        dict_player['Stats'] = summary_table
        dict_contracts[p] = dict_player
        
    return dict_contracts

In [99]:
# get player data for new contract players
dict_new_contracts = getPlayerContractData_newContracts(df_players_new,2023)

In [101]:
# construct dataframe with statistics for new contract players
dataTable_ng_new = getDataTable_ng(dict_new_contracts)

In [103]:
dataTable_ng_new.head()

,Name,ContractAge,Position,Nationality,Height,Overall Pick,GAMES_PLAYED_t,G_t,A_t,P_t,...,ixG_PLAYOFF_avg,xG±/60_PLAYOFF_avg,RelxG±/60_PLAYOFF_avg,C±/60_PLAYOFF_avg,RelC±/60_PLAYOFF_avg,G_PG_avg,P_PG_avg,PIM_PG_avg,G_PG_PLAYOFF_avg,P_PG_PLAYOFF_avg
2023,Zach Parise,39.0,L,USA,71.0,17.0,82.0,21.0,13.0,34.0,...,0.563333,-0.200000,0.073333,-14.293333,-6.610000,0.198193,0.413821,0.255827,0.166667,0.250000
2023,Damon Severson,29.0,D,CAN,74.0,60.0,81.0,7.0,26.0,33.0,...,0.110000,0.073333,-0.016667,1.773333,1.770000,0.092497,0.452469,0.566498,0.027778,0.083333
2023,John Moore,33.0,D,USA,75.0,21.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.271429,0.485714,0.000000,0.000000
2023,Miles Wood,28.0,L,USA,74.0,100.0,76.0,13.0,14.0,27.0,...,0.310000,-0.043333,-0.086667,-6.646667,-5.193333,0.160048,0.269936,0.953535,0.083333,0.083333
2023,Steven Santini,28.0,D,USA,74.0,42.0,4.0,0.0,1.0,1.0,...,0.050000,0.630000,1.353333,29.753333,37.203333,0.000000,0.194444,0.166667,0.000000,0.000000


In [104]:
dataTable_ng_new.shape

(184, 122)

It is important to note that of the 267 free agents we first scraped the names of, we only recover data for 184 of them - this is because of the free agents that are technically available, it is likely that some of them have not played in the last three years or have been playing in the minor league for that period while technically on a major team contract. Since we do not have adequate information for those players, they are excluded.

In [109]:
# save new data
dataTable_ng_new.to_csv('new_contract_players.csv',index=False)